# HW 04 - Applied ML

In [33]:
import pandas as pd
import sklearn

from sklearn_pandas import DataFrameMapper

In [39]:
df = pd.read_csv('CrowdstormingDataJuly1st.csv', index_col='playerShort')
df.sample(10)

,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
playerShort,,,,,,,,,,,,,,,,,,,,,
richard-dunne,Richard Dunne,Aston Villa,England,21.09.1979,188.0,95.0,Center Back,4,3,0,...,0.00,1612,93,WALES,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037
dennis-diekmeier,Dennis Diekmeier,Hamburger SV,Germany,20.10.1989,188.0,79.0,Right Fullback,1,1,0,...,0.25,1019,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225
farinos,Farinós,Villarreal CF,Spain,29.03.1978,173.0,66.0,Defensive Midfielder,3,2,1,...,NaN,270,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
nicky-shorey,Nicky Shorey,Reading FC,England,19.02.1981,175.0,67.0,Left Fullback,1,0,0,...,0.00,552,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037
steven-whittaker,Steven Whittaker,Norwich City,England,16.06.1984,186.0,87.0,Right Fullback,1,1,0,...,0.25,1956,40,SWE,0.340205,5223.0,0.000081,0.626401,5621.0,0.000373
garrido_2,Garrido,Norwich City,England,15.03.1985,178.0,75.0,Left Fullback,5,1,2,...,0.25,641,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
matthieu-valbuena,Matthieu Valbuena,Olympique Marseille,France,28.09.1984,167.0,58.0,Right Midfielder,3,2,1,...,0.00,362,7,FRA,0.334684,2882.0,0.000151,0.336101,3011.0,0.000586
hiroshi-kiyotake,Hiroshi Kiyotake,1. FC Nürnberg,Germany,12.11.1989,172.0,63.0,Attacking Midfielder,3,2,0,...,0.50,1965,41,JPN,0.341308,2899.0,0.000147,0.205339,3034.0,0.000603
papiss-demba-cisse,Papiss Demba Cissé,Newcastle United,England,03.06.1985,183.0,73.0,Center Forward,2,1,0,...,1.00,2536,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225


In [40]:
df.columns

Index(['player', 'club', 'leagueCountry', 'birthday', 'height', 'weight',
       'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'photoID', 'rater1', 'rater2',
       'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp'],
      dtype='object')

In [41]:
# Remove columns with no rating
df = df.dropna(subset=['rater1', 'rater2'])
df['rating'] = df.rater1 + df.rater2

lim = df.drop(['player',
               'club',
               'leagueCountry',
               'birthday',
               'height',
               'weight',
               'position',
               'rater1',
               'rater2',
               'Alpha_3',
               'photoID'], axis=1)

lim.sample(10)

,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,rating
playerShort,,,,,,,,,,,,,,,,,
carlos-tevez,1,0,0,1,0,0,0,0,2986,44,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,1.00
steven-whittaker,8,7,0,1,3,1,0,0,74,45,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.25
tiago_3,1,1,0,0,0,0,0,0,737,48,0.386174,1761.0,0.000232,0.529815,1895.0,0.001091,0.00
lewis-holtby,1,1,0,0,0,0,0,0,498,8,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225,0.00
robert-green,4,2,1,1,0,0,0,1,554,44,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.50
tripy-makonda,5,1,1,3,0,2,0,0,548,7,0.334684,2882.0,0.000151,0.336101,3011.0,0.000586,1.75
oriol-romeu,1,1,0,0,0,0,0,0,68,28,0.355498,4078.0,0.000098,0.517225,4238.0,0.000405,0.50
karim-guede,1,1,0,0,0,0,0,0,2885,8,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225,1.50
christophe-jallet,1,0,0,1,0,0,0,0,3025,21,0.376127,574.0,0.000714,0.498350,606.0,0.002968,0.50


In [46]:
lim[lim.redCards == 2]

,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,rating
playerShort,,,,,,,,,,,,,,,,,
bodipo,6,2,1,3,1,0,0,2,140,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.75
antonio-lopez_2,8,5,2,1,0,2,0,2,367,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.50
javi-martinez,14,4,3,7,2,2,0,2,432,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.00
jonas,9,1,4,4,1,0,0,2,432,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.50
phil-jagielka,7,2,1,4,1,0,0,2,487,44,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.50
cyril-jeunechamp,14,8,0,6,0,6,0,2,586,7,0.334684,2882.0,0.000151,0.336101,3011.0,0.000586,0.00
sergio-ramos,18,12,1,5,4,6,1,2,804,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.00
aly-cissokho,9,1,5,3,1,1,0,2,985,7,0.334684,2882.0,0.000151,0.336101,3011.0,0.000586,2.00
eugen-polanski,8,4,0,4,0,0,0,2,1114,8,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225,0.00
